In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
plt.rcParams["figure.figsize"] = [12,12]
def fix_scaling(ax=None):
    if not ax:
        xlim = plt.xlim()
        ylim = plt.ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            plt.ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            plt.xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))
    else:
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            ax.set_ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            ax.set_xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))

In [2]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
def animate_trajectory(t1, A, b, c):
    fig, ax = plt.subplots()
    n = len(t1)
    def step(t):
        ax.cla()
        ax.plot([3.0 / 4], [1], 'o', color='yellow')
        #Level contours
        delta = 0.025
        x = np.arange(-3, 5, delta)
        y = np.arange(-3, 5, delta)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                Z[i][j] = func(A, b, c, [X[i][j], Y[i][j]])
        CS = ax.contour(X, Y, Z, [1, 3, 6], colors=['blue', 'purple', 'red'])
        ax.plot([1, 1, 0, 0, 1], [1, 0, 0, 1, 1], color='green')
        ax.plot([u[0] for u in t1[:t + 1]], [u[1] for u in t1[:t + 1]], color='black')
        ax.plot([u[0] for u in t1[:t + 1]], [u[1] for u in t1[:t + 1]], 'o', color='black')
        
        fix_scaling(ax)
        ax.axis('off')
    plt.close()
    return FuncAnimation(fig, step,
                     frames=range(n), interval=600)

In [3]:
def func(A, b, c, x):
    return 0.5 * np.matmul(np.matmul(x, A), np.transpose(x)) - np.matmul(b, np.transpose(x)) + c
    
def f_grad(A, b, x):
    return np.transpose(np.matmul(A, np.transpose(x)) - np.transpose(b))
    
def f_grad2(A):
    return A

In [4]:
N = 60
def gradient_proj(x_start, A, b, c):
    alpha = 2.0 / sum(list(np.linalg.eig(f_grad2(A))[0]))
    cur_x = x_start.copy()
    traj = []
    traj.append(cur_x.copy())
    
    for i in range(N):
        cur_x = cur_x - alpha * f_grad(A, b, cur_x)
        cur_x = map(lambda x: min(1, max(0, x)), cur_x)
        
        traj.append(cur_x)
        
    return traj

In [5]:
# n=3
A = np.array([[4, -2, 3], [-2, 2, 2], [-1, 1, 2]])
b = np.array([1, 0, 0])
c = 0
ans = [0.5, 0.5, 0]
print(gradient_proj(np.array([1, 1, 1]), A, b, c)[-1])
print(ans)

[0.4934047088995619, 0.4903438236475449, 0.0017672029242105558]
[0.5, 0.5, 0]


In [32]:
# n=2
a1 = np.array([[4, -2], [-2, 2]])
b1 = np.array([1, 2])
c = 0
traj = gradient_proj(np.array([1, 1]), a1, b1, c)
base_animation = animate_trajectory(traj, a1, b1, c)
HTML(base_animation.to_html5_video())